# Mutual Fund Volatility Analyzer


In [2]:
import pandas as pd

# Load your original file (semicolon separated)
file_path = "DownloadNAVHistoryReport_Po.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8', on_bad_lines='skip')

# Strip column names of any extra spaces
df.columns = df.columns.str.strip()

In [3]:

# Drop rows where scheme code is missing
df = df[pd.to_numeric(df['Scheme Code'], errors='coerce').notnull()]
df['Scheme Code'] = df['Scheme Code'].astype(int)

# Keep only Scheme Code and Scheme Name
df_scheme = df[['Scheme Code', 'Scheme Name']].drop_duplicates()

In [4]:
# Save to new CSV file
df_scheme.to_csv("Schemes_List.csv", index=False)

print(" Extracted scheme codes and names successfully!")
print(df_scheme.head())
print("Saved as 'Schemes_List.csv'")


 Extracted scheme codes and names successfully!
   Scheme Code                                        Scheme Name
2       139619  Taurus Investor Education Pool - Unclaimed Div...
3       139618  Taurus Investor Education Pool - Unclaimed Red...
4       139617               Taurus Unclaimed Redemption - Growth
7       148921  Aditya Birla Sun Life Multi-Cap Fund-Direct Gr...
8       148920  Aditya Birla Sun Life Multi-Cap Fund-Direct ID...
Saved as 'Schemes_List.csv'


In [5]:
import requests
response = requests.get("https://api.mfapi.in/mf/133895", timeout=10)
print(response.status_code)
print(response.json())


200
{'meta': {'fund_house': 'Axis Mutual Fund', 'scheme_type': 'Open Ended Schemes', 'scheme_category': 'Debt Scheme - Short Duration Fund', 'scheme_code': 133895, 'scheme_name': 'Axis Short Duration Fund - Regular Plan - Regular IDCW', 'isin_growth': 'INF846K01RJ1', 'isin_div_reinvestment': 'INF846K01RK9'}, 'data': [{'date': '30-07-2025', 'nav': '20.88130'}, {'date': '29-07-2025', 'nav': '20.88090'}, {'date': '28-07-2025', 'nav': '20.88100'}, {'date': '25-07-2025', 'nav': '20.87500'}, {'date': '24-07-2025', 'nav': '20.88150'}, {'date': '23-07-2025', 'nav': '20.88010'}, {'date': '22-07-2025', 'nav': '20.87550'}, {'date': '21-07-2025', 'nav': '20.86690'}, {'date': '18-07-2025', 'nav': '20.85710'}, {'date': '17-07-2025', 'nav': '20.84830'}, {'date': '16-07-2025', 'nav': '20.83950'}, {'date': '15-07-2025', 'nav': '20.83910'}, {'date': '14-07-2025', 'nav': '20.83590'}, {'date': '11-07-2025', 'nav': '20.82800'}, {'date': '10-07-2025', 'nav': '20.82450'}, {'date': '09-07-2025', 'nav': '20.82

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import time

def fetch_nav_data(scheme_code, start_date, end_date):
    url = f"https://api.mfapi.in/mf/{scheme_code}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            navs = data.get("data", [])
            nav_df = pd.DataFrame(navs)
            nav_df["date"] = pd.to_datetime(nav_df["date"], format="%d-%m-%Y")
            nav_df["nav"] = pd.to_numeric(nav_df["nav"], errors="coerce")
            nav_df = nav_df[(nav_df["date"] >= start_date) & (nav_df["date"] <= end_date)]
            return nav_df.sort_values(by="date", ascending=True)
        else:
            print(f"Error fetching data for scheme code {scheme_code}")
            return None
    except Exception as e:
        print(f"Exception for scheme {scheme_code}: {e}")
        return None

def calculate_volatility(nav_df):
    if len(nav_df) < 2:
        return None
    returns = nav_df["nav"].pct_change().dropna()
    return returns.std()

def calculate_return(nav_df):
    if len(nav_df) < 2:
        return None
    start_nav = nav_df["nav"].iloc[0]
    end_nav = nav_df["nav"].iloc[-1]
    return ((end_nav - start_nav) / start_nav) * 100

def categorize_risk(volatility):
    if pd.isna(volatility):
        return "Unknown"
    elif volatility < 0.005:
        return "Low Risk"
    elif volatility < 0.015:
        return "Medium Risk"
    else:
        return "High Risk"

# Load your input file (CSV must contain: Scheme Code, Scheme Name)
input_file = "Schemes_List.csv"
df_input = pd.read_csv(input_file)

# Date range for last 90 days
end_date = datetime.today()
start_date = end_date - timedelta(days=90)

results = []

for idx, row in df_input.iterrows():
    scheme_code = row["Scheme Code"]
    scheme_name = row["Scheme Name"]

    print(f"Fetching NAV for: {scheme_name} ({scheme_code})")

    nav_data = fetch_nav_data(scheme_code, start_date, end_date)

    if nav_data is not None and len(nav_data) > 0:
        volatility = calculate_volatility(nav_data)
        returns = calculate_return(nav_data)
        risk = categorize_risk(volatility)

        results.append({
            "Scheme Code": scheme_code,
            "Scheme Name": scheme_name,
            "Volatility (90d)": round(volatility, 6) if pd.notna(volatility) else None,
            "Return (90d%)": round(returns, 2) if pd.notna(returns) else None,
            "Risk_Category": risk
        })
    else:
        print(f"Not enough data for {scheme_name}")

    time.sleep(1)  # Respectful delay to avoid rate-limiting

# Save output
output_df = pd.DataFrame(results)
output_df.to_csv("mutual_fund_volatility_report.csv", index=False)
print("\n Report generated: mutual_fund_volatility_report.csv")

Fetching NAV for: Taurus Investor Education Pool - Unclaimed Dividend - Growth (139619)
Fetching NAV for: Taurus Investor Education Pool - Unclaimed Redemption - Growth (139618)
Fetching NAV for: Taurus Unclaimed Redemption - Growth (139617)
Fetching NAV for: Aditya Birla Sun Life Multi-Cap Fund-Direct Growth (148921)
Fetching NAV for: Aditya Birla Sun Life Multi-Cap Fund-Direct IDCW Payout (148920)
Fetching NAV for: Aditya Birla Sun Life Multi-Cap Fund-Regular Growth (148918)
Fetching NAV for: Aditya Birla Sun Life Multi-Cap Fund-Regular-IDCW Payout (148919)
Fetching NAV for: Axis Multicap Fund - Direct Growth (149383)
Fetching NAV for: Axis Multicap Fund - Direct Plan - IDCW (149384)
Fetching NAV for: Axis Multicap Fund - Regular Plan - Growth (149382)
Fetching NAV for: Axis Multicap Fund - Regular Plan - IDCW (149387)
Fetching NAV for: BAJAJ FINSERV MULTI CAP FUND - DIRECT - GROWTH (153309)
Fetching NAV for: BAJAJ FINSERV MULTI CAP FUND - DIRECT - IDCW (153306)
Fetching NAV for: BAJ

C:\Users\mruna\AppData\Local\Temp\ipykernel_26816\854892356.py:36: RuntimeWarning: invalid value encountered in scalar divide
  return ((end_nav - start_nav) / start_nav) * 100


Fetching NAV for: PGIM India Money Market Fund - Segregated Portfolio 1 - Direct Plan - Growth Option (152114)


C:\Users\mruna\AppData\Local\Temp\ipykernel_26816\854892356.py:36: RuntimeWarning: invalid value encountered in scalar divide
  return ((end_nav - start_nav) / start_nav) * 100


Fetching NAV for: PGIM India Money Market Fund - Segregated Portfolio 1 - Direct Plan - Monthly IDCW Option (152112)


...

Fetching NAV for: Nippon India Equity Savings Fund - Segregated Portfolio 2 - Growth Plan - Bonus Option (148279)


C:\Users\mruna\AppData\Local\Temp\ipykernel_26816\854892356.py:36: RuntimeWarning: invalid value encountered in scalar divide
  return ((end_nav - start_nav) / start_nav) * 100


Fetching NAV for: Nippon India Equity Savings Fund - Segregated Portfolio 2 - Growth Plan - Growth Option (148280)


C:\Users\mruna\AppData\Local\Temp\ipykernel_26816\854892356.py:36: RuntimeWarning: invalid value encountered in scalar divide
  return ((end_nav - start_nav) / start_nav) * 100


Fetching NAV for: Nippon India Equity Savings Fund- Direct Plan- Growth Plan- Bonus Option (134595)
Fetching NAV for: Nippon India Equity Savings Fund- Direct Plan- Growth Plan-Growth Option (134594)
Fetching NAV for: Nippon India Equity Savings Fund- Growth Plan- Bonus Option (134602)
Fetching NAV for: Nippon India Equity Savings Fund- Growth Plan- Growth Option (134593)
Fetching NAV for: PGIM India Equity Savings Fund - Annual Dividend option (138371)
Fetching NAV for: PGIM India Equity Savings Fund - Direct Plan - Annual Dividend (138375)
Fetching NAV for: PGIM India Equity Savings Fund - Direct Plan - Growth (138376)
Fetching NAV for: PGIM India Equity Savings Fund - Direct Plan - Monthly Dividend (138377)
Fetching NAV for: PGIM India Equity Savings Fund - Growth Option (138372)
Fetching NAV for: PGIM India Equity Savings Fund - Monthly Dividend option (138369)
Fetching NAV for: SBI Equity Savings Fund - Direct Plan - Growth (134643)
Fetching NAV for: SBI Equity Savings Fund - Dire

In [ ]:
import os

output_folder = "mutualfund_data_final"
os.makedirs(output_folder, exist_ok=True)

output_path = os.path.join(output_folder, "mutual_fund_volatility_report.csv")
output_df.to_csv(output_path, index=False)

print(f"\n Report saved to: {output_path}")
